
# Forensics Challenge 1 - The Outlook Web Access Attack

The victim lost 1.2 million dollars in an Outlook Web Access Phishing attack. You have pulled all of the SUCCESSFUL LOGINS from the Outlook Administrator console. But there are THOUSANDS of IP addresses to look through. The original attack came from Iran. You decide to focus your attention on those requests that are from the same country of origin.

```
['144.12.97.175',
 '47.33.190.195',
 '54.213.136.220',
 '37.27.54.15',
 '91.107.252.94',
 '112.245.28.187',
 '77.198.208.109',
 '79.195.246.215',
 '109.219.196.231',
 '111.14.43.156',
 '102.111.174.6',
 '176.65.253.46',
 '102.155.114.82']
 ```

Which IP Addresses originated in Iran?

## Solutions

One easy solution for this is to use the code we looked at in notebook title "1 - IP Geolocation with Websites".



In [ ]:
import requests

browser = requests.session()
list_of_addresses = ['144.12.97.175', '47.33.190.195', '54.213.136.220', '37.27.54.15', '91.107.252.94', '112.245.28.187', '77.198.208.109', '79.195.246.215', '109.219.196.231', '111.14.43.156', '102.111.174.6', '176.65.253.46', '102.155.114.82']
for each_ip in list_of_addresses:
    url = f"https://freegeoip.app/json/{each_ip}"
    result = browser.get(url).json()
    print(f"The country for IP Address {result.get('ip')} is {result.get('country_name')}")

# Forensics Challenge 2 - At the scene of the crime

After obtaining the suspect's laptop you need to place him at the scene of the crime.  After pulling their wireless history from the registry, SRUM and Windows Event logs you come up with the following list of Wireless BSSIDs.  What locations can you place the suspects laptop at from the following Wireless BSSID addresses?

```
['70-0F-6A-32-2C-0F',
'0C-F5-A4-96-CD-C7',
'74-3E-2B-36-57-E8',
'08-4F-A9-3C-F5-A0',
'70-70-8B-29-12-81',
'0C-F5-A4-96-CD-CB']
```

## Solution

Here is one way you could solve this one. We combine various techniques used in notebook "4 - Geolocating Multiple Access Points".

In [ ]:
import requests
import itertools
    
key = 'copy and paste mark baggetts API key here'

list_of_aps = ['70-0F-6A-32-2C-0F',
'0C-F5-A4-96-CD-C7',
'74-3E-2B-36-57-E8',
'08-4F-A9-3C-F5-A0',
'70-70-8B-29-12-81',
'0C-F5-A4-96-CD-CB']

def find_location(list_of_aps, key):
    url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={key}' 
    ap_list = []
    for mac in list_of_aps:
        ap_list.append({"macAddress":mac ,"signalStrength":"-60"})
    google_data = {"considerIP": "false", "wifiAccessPoints": ap_list}
    response = requests.post(url=url, json=google_data)    
    loc_record = response.json()
    lat = loc_record.get("location").get("lat")
    lng = loc_record.get("location").get("lng")
    accuracy = loc_record.get('accuracy')
    return accuracy, f"http://maps.google.com/maps?q={lat:0>3.9f},{lng:0>3.9f}&z=15"

my_location = find_location(['00-00-00-00-00-00','11-11-11-11-11-11'],key)
for each_pair in itertools.combinations(list_of_aps,2):
    location = find_location(each_pair, key)
    if location != my_location:
        print(f"WIFI {str(each_pair)}  Accuracy {location[0]}. Link {location[1]}")
    else:
        print(f"WIFI {str(each_pair)} not found")
